# Fashion MNIST DCGAN

This DCGAN is loosely based on various tutorials I have found online. 


In [ ]:
import pickle as pkl
import tensorflow as tf
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
%matplotlib inline


## Set an optional output directory if you want to keep some in-progress generative images

In [ ]:
save_output = False

if save_output:
    !mkdir output
    output_dir = 'output'


## Getting the dataset

You can download and unpack the FashionMNIST dataset here:

https://github.com/zalandoresearch/fashion-mnist

I suggest leaving it where it checks out, and modifying the path in the next cell. If you checked it out in this directory, the path below should work out of the box.

Since FashionMNIST is a drop in replacement for MNIST, we can lean on the tensorflow mechanisms for train/test and batch generation for the rest of the notebook. Its pretty convenient.

## if you run the next cell before you download the FashionMNIST dataset, it will download regular MNIST and you'll just get numbers


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('fashion-mnist/data/fashion', one_hot=True)

## Check out a random sampling of the images

In [ ]:

idx = np.random.randint(0, 1000, size=36)
print(idx)
fig, axes = plt.subplots(6, 6, sharex=True, sharey=True, figsize=(5,5),)
for ii, ax in zip(idx, axes.flatten()):
    ax.imshow(data.train.images[ii].reshape(28,28), aspect='equal', cmap='gray')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
def scale(x, feature_range=(-1, 1)):
    # scale to (0, 1)
    x = ((x - x.min())/(255 - x.min()))
    
    # scale to feature_range
    min, max = feature_range
    x = x * (max - min) + min
    return x


## Generator
Builds up from the noise input. 




In [ ]:
def generator(z, output_dim, reuse=False, alpha=0.2, training=True):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        x1 = tf.layers.dense(z, 7*7*64)
        # Reshape it to start the convolutional stack
        x1 = tf.reshape(x1, (-1, 7, 7, 64))
        x1 = tf.layers.batch_normalization(x1, training=training)
        x1 = tf.maximum(alpha * x1, x1)
        # 7x7x64 now

        x2 = tf.layers.conv2d_transpose(x1, 32, 5, strides=2, padding='same')
        x2 = tf.layers.batch_normalization(x2, training=training)
        x2 = tf.maximum(alpha * x2, x2)
        # 14x14x32 now
        
        x3 = tf.layers.conv2d_transpose(x2, 16, 5, strides=2, padding='same')
        x3 = tf.layers.batch_normalization(x3, training=training)
        x3 = tf.maximum(alpha * x3, x3)
        # 28x28x16 now
        
        # Output layer
        logits = tf.layers.conv2d_transpose(x3, 1, 5, strides=1, padding='same')
        # 28x28x1 now
        
        out = tf.tanh(logits)
        
        return out

## Discriminator

This is basically just a convolutional classifier. The input to the discriminator are 28x28x1 images.

Note that it hasn't been updated with the latest findings, and sticks closer to the DCGAN paper.

In [ ]:
def discriminator(x, reuse=False, alpha=0.2):
    with tf.variable_scope('discriminator', reuse=reuse):

        d_x_image = tf.reshape(x, [-1,28,28,1])
        
        x1 = tf.layers.conv2d(d_x_image, 16, 5, strides=2, padding='same')
        relu1 = tf.maximum(alpha * x1, x1)

        x2 = tf.layers.conv2d(relu1, 32, 5, strides=2, padding='same')
        bn2 = tf.layers.batch_normalization(x2, training=True)
        relu2 = tf.maximum(alpha * bn2, bn2)

        x3 = tf.layers.conv2d(relu2, 64, 5, strides=1, padding='same')
        bn3 = tf.layers.batch_normalization(x3, training=True)
        relu3 = tf.maximum(alpha * bn3, bn3)

        flat = tf.reshape(relu3, (-1, 7*7*64))
        logits = tf.layers.dense(flat, 1)
        out = tf.sigmoid(logits)
        
        return out, logits

## Model Loss

There are a couple loss functions in here. Real images, generated images, and the fake one.

In [ ]:
def model_loss(input_real, input_z, output_dim, alpha=0.2):
   
    g_model = generator(input_z, output_dim, alpha=alpha)
    d_model_real, d_logits_real = discriminator(input_real, alpha=alpha)
    d_model_fake, d_logits_fake = discriminator(g_model, reuse=True, alpha=alpha)

    d_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_model_real)))
    d_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_model_fake)))
    g_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_model_fake)))

    d_loss = d_loss_real + d_loss_fake

    return d_loss, g_loss

## Building the model



In [ ]:
class GAN:
    def __init__(self, real_size, z_size, learning_rate, alpha=0.2, beta1=0.5):
        tf.reset_default_graph()
        
        self.input_real = tf.placeholder(tf.float32, (None, 28, 28), name='input_real')
        self.input_z = tf.placeholder(tf.float32, (None, z_size), name='input_z')
        
        self.d_loss, self.g_loss = model_loss(self.input_real, self.input_z,
                                              784, alpha=alpha)
        
        # Get weights and bias to update
        self.t_vars = tf.trainable_variables()
        self.d_vars = [var for var in self.t_vars if var.name.startswith('discriminator')]
        self.g_vars = [var for var in self.t_vars if var.name.startswith('generator')]

        # Build the independent Optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.d_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(self.d_loss, var_list=self.d_vars)
            self.g_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(self.g_loss, var_list=self.g_vars)



Here is a function for displaying generated images. I use it inline or save the figure to the output dir

In [ ]:
def view_samples(epoch, samples, nrows, ncols, figsize=(5,5)):
    fig, axes = plt.subplots(figsize=figsize, nrows=nrows, ncols=ncols, 
                             sharey=True, sharex=True)
    for ax, img in zip(axes.flatten(), samples[epoch]):
        ax.axis('off')
        img = ((img - img.min())*255 / (img.max() - img.min())).astype(np.uint8)
        ax.set_adjustable('box-forced')
        im = ax.imshow(img.reshape(28, 28), aspect='equal', cmap="gray")
   
    plt.subplots_adjust(wspace=0, hspace=0)
    return fig, axes

In [ ]:
def train(net, dataset, epochs, batch_size, figsize=(5,5)):
    saver = tf.train.Saver()
    sample_z = np.random.uniform(-1, 1, size=(72, z_size))

    #capture a couple images along the way, and grab the losses so we can 
    #Graph them.
    samples, losses = [], []
    steps = 0

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for e in range(epochs):
            for i in range(1000):
                b, y = data.train.next_batch(batch_size)
                x = b.reshape((batch_size, 28,28))
                steps += 1

                # Sample random noise for G
                batch_z = np.random.uniform(-1, 1, size=(batch_size, z_size))

                # Run optimizers
                
                _ = sess.run(net.d_opt, feed_dict={net.input_real: x, net.input_z: batch_z})
                _ = sess.run(net.g_opt, feed_dict={net.input_z: batch_z, net.input_real: x})

                if steps % 500 == 0:
                    # At the end of each epoch, get the losses and print them out
                    train_loss_d = net.d_loss.eval({net.input_z: batch_z, net.input_real: x})
                    train_loss_g = net.g_loss.eval({net.input_z: batch_z})

                    print("Epoch {}/{}...".format(e+1, epochs),
                          "Discriminator Loss: {:.4f}...".format(train_loss_d),
                          "Generator Loss: {:.4f}".format(train_loss_g))
                    # Save losses to view after training
                    losses.append((train_loss_d, train_loss_g))

                # CHANGE THIS IF YOU WANT TO SEE MORE OR LESS FREQUENT IMAGES
                if steps % 500 == 0:
                    gen_samples = sess.run(generator(net.input_z, 3, reuse=True, training=False),
                                   feed_dict={net.input_z: sample_z})
                    samples.append(gen_samples)
                    fig, _ = view_samples(-1, samples, 6, 12, figsize=figsize)
                    if save_output:
                        fig.savefig("./" + output_dir + '/' + str(e) + "_" + str(steps) + "_img.png")
                    else:
                        plt.show()
                    plt.close(fig)

        saver.save(sess, './fashion/generator.ckpt')

    with open('samples.pkl', 'wb') as f:
        pkl.dump(samples, f)
    
    return losses, samples

## Train

GANs are very sensitive to hyperparameters.
Check out [the DCGAN paper](https://arxiv.org/pdf/1511.06434.pdf) - these are rough approximations.

In [ ]:
real_size = (28, 28)
z_size = 100
learning_rate = 0.0002
batch_size = 128
epochs = 25
alpha = 0.2
beta1 = 0.5

# Create & train the network
net = GAN(real_size, z_size, learning_rate, alpha=alpha, beta1=beta1)
losses, samples = train(net, None, epochs, batch_size, figsize=(10,5))

In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()

In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()

In [ ]:
_ = view_samples(-1, samples, 6, 12, figsize=(10,5))